In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Evaluate Llama 3.1 models using Vertex AI AutoSxS

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_autosxs_evaluation_llama3_1.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_autosxs_evaluation_llama3_1.ipynb"">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_autosxs_evaluation_llama3_1.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_autosxs_evaluation_llama3_1.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook demostrates how to use the Vertex AI automatic side-by-side (AutoSxS) tool to evaluate Llama 3.1 models for a question-answering task.

### Objective

- Choose the Llama 3.1 models you want to compare.

- Create an evaluation dataset with question-answer data.

- Create and run a Vertex AI AutoSxS pipeline that generates judgments and a set of AutoSxS metrics using the generated judgments.

- Print the judgments and AutoSxS metrics.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform google-cloud-pipeline-components
! pip3 install --upgrade --user --quiet openai gcsfs

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "<your-project-id>"  # @param {type:"string"}

# Set the region of the instance
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store tutorial artifacts.

In [ ]:
BUCKET_NAME = "<your-bucket-name>"  # @param {type:"string"}

BUCKET_URI = f"gs://{BUCKET_NAME}"

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### Initialize Vertex AI SDK for Python

In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import libraries

Import libraries to use in this tutorial.

In [ ]:
import random
import string

import openai
import pandas as pd
from google.auth import default, transport
from google.cloud import aiplatform
from google_cloud_pipeline_components.v1 import model_evaluation
from kfp import compiler

### Set variables

Before starting, you must decide how to access Llama 3.1 models. You can access Llama 3.1 models in just a few clicks using Model-as-a-Service (MaaS) without any setup or infrastructure hassles. You can also access Llama models for self-service in Vertex AI Model Garden, allowing you to choose your preferred infrastructure.

This tutorial assumes that you deploy a self-managed instance of the Llama 3.1 model and compare it with Llama 3 405b using Model-as-a-Service (MaaS). Notice, only `us-central1` is supported region for Llama 3.1 models using Model-as-a-Service (MaaS).

[Check out Llama 3 model card](https://console.cloud.google.com/vertex-ai/publishers/meta/model-garden/llama3?_ga=2.31261500.2048242469.1721714335-1107467625.1721655511) to learn how to deploy a Llama 3.1 models on Vertex AI.

In [ ]:
SELF_DEPLOYED_ENDPOINT_REGION = "<your-endpoint-region>"  # @param {type:"string"}
SELF_DEPLOYED_ENDPOINT_ID = "<your-endpoint-id>"  # @param {type:"string"}
MODEL_LOCATION = "us-central1"

### Helpers

In [ ]:
def self_model_generate(
    question,
    context,
    endpoint_id=SELF_DEPLOYED_ENDPOINT_ID,
    endpoint_location=SELF_DEPLOYED_ENDPOINT_REGION,
    **model_kwargs,
):
    """Generate a response from a self-managed Llama 3.1 model."""

    aiplatform.init(project=PROJECT_ID, location=endpoint_location)

    prompt = """You are an AI assistant. Your goal is to answer questions using the pieces of context. """
    prompt += f"""Question: {question}."""
    prompt += f"""Context: {context}."""
    prompt += """Answer:"""

    instance = {"prompt": prompt}
    instance.update(model_kwargs)
    instances = [instance]

    endpoint = aiplatform.Endpoint(endpoint_id)
    response = endpoint.predict(instances=instances)
    return response.predictions[0][len(prompt) + 1 :]


def maas_generate(
    question,
    context,
    model="meta/llama3-405b-instruct-maas",
    model_location=MODEL_LOCATION,
    **model_kwargs,
):
    """Generate a response from a MaaS Llama 3.1 model."""

    creds, _ = default()
    auth_req = transport.requests.Request()
    creds.refresh(auth_req)
    if model_kwargs is None:
        model_kwargs = {}

    client = openai.OpenAI(
        base_url=f"https://{model_location}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{model_location}/endpoints/openapi/chat/completions?",
        api_key=creds.token,
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": """You are an AI assistant. Your goal is to answer questions using the pieces of context. If you don't know the answer, say that you don't know.""",
            },
            {"role": "user", "content": question},
            {"role": "assistant", "content": context},
        ],
        **model_kwargs,
    )

    return response.choices[0].message.content


def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specified length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

### Generate evaluation dataset for AutoSxS

Below you create your evaluation dataset, you specify a set of prompts to evaluate on.

In this notebook, you:

- Use 10 examples from the original dataset to create an evaluation dataset for AutoSxS.
  - Data in the `contexts` column will be treated as model context.
  - Data in the `questions` column will be treated as model instruction.
  - Data in the `response_a` column will be treated as responses for model A.
  - Data in the `response_b` will be treated as responses for model B.

- Store the data in a JSON file in Google sCloud Storage.

#### **Note: For the best results we recommend using at least 100 examples. There are diminishing returns when using more than 400 examples.**

#### Provide context and question

In [ ]:
contexts = [
    "Beginning in the late 1910s and early 1920s, Whitehead gradually turned his attention from mathematics to philosophy of science, and finally to metaphysics. He developed a comprehensive metaphysical system which radically departed from most of western philosophy. Whitehead argued that reality consists of processes rather than material objects, and that processes are best defined by their relations with other processes, thus rejecting the theory that reality is fundamentally constructed by bits of matter that exist independently of one another. Today Whitehead's philosophical works – particularly Process and Reality – are regarded as the foundational texts of process philosophy.",
    "The gills have an adnate attachment to the cap, are narrow to moderately broad, closely spaced, and eventually separate from the stem. Young gills are cinnamon-brown in color, with lighter edges, but darken in maturity because they become covered with the dark spores. The stem is 6 to 8 cm (2+3⁄8 to 3+1⁄8 in) long by 1.5 to 2 mm (1⁄16 to 3⁄32 in) thick, and roughly equal in width throughout except for a slightly enlarged base. The lower region of the stem is brownish in color and has silky 'hairs' pressed against the stem; the upper region is grayish and pruinose (lightly dusted with powdery white granules). The flesh turns slightly bluish or greenish where it has been injured. The application of a drop of dilute potassium hydroxide solution on the cap or flesh will cause a color change to pale to dark yellowish to reddish brown; a drop on the stem produces a less intense or no color change.",
    "Go to Device Support. Choose your device. Scroll to Getting started and select Hardware & phone details. Choose Insert or remove SIM card and follow the steps. Review the Account Summary page for details. Image 13 Activate online Go to att.com/activateprepaid ((att.com/activarprepaid for Spanish)) and follow the prompts. Activate over the phone Call us at 877.426.0525 for automated instructions. You will need to know your SIM/eSIM ICCID & IMEI number for activation. Note: Look for your SIM (( ICCID )) number on your box or SIM card Now youre ready to activate your phone 1. Start with your new device powered off. 2. To activate a new line of service or a replacement device, please go to the AT&T Activation site or call 866.895.1099. You download the eSIM to your device over Wi-Fi®. The eSIM connects your device to our wireless network. How do I activate my phone with an eSIM? Turn your phone on, connect to Wi-Fi, and follow the prompts. Swap active SIM cards AT&T Wireless SM SIM Card Turn your device off. Remove the old SIM card. Insert the new one. Turn on your device.",
    "According to chief astronaut Deke Slayton's autobiography, he chose Bassett for Gemini 9 because he was 'strong enough to carry' both himself and See. Slayton had also assigned Bassett as command module pilot for the second backup Apollo crew, alongside Frank Borman and William Anders.",
    "Adaptation of the endosymbiont to the host's lifestyle leads to many changes in the endosymbiont–the foremost being drastic reduction in its genome size. This is due to many genes being lost during the process of metabolism, and DNA repair and recombination. While important genes participating in the DNA to RNA transcription, protein translation and DNA/RNA replication are retained. That is, a decrease in genome size is due to loss of protein coding genes and not due to lessening of inter-genic regions or open reading frame (ORF) size. Thus, species that are naturally evolving and contain reduced sizes of genes can be accounted for an increased number of noticeable differences between them, thereby leading to changes in their evolutionary rates. As the endosymbiotic bacteria related with these insects are passed on to the offspring strictly via vertical genetic transmission, intracellular bacteria goes through many hurdles during the process, resulting in the decrease in effective population sizes when compared to the free living bacteria. This incapability of the endosymbiotic bacteria to reinstate its wild type phenotype via a recombination process is called as Muller's ratchet phenomenon. Muller's ratchet phenomenon together with less effective population sizes has led to an accretion of deleterious mutations in the non-essential genes of the intracellular bacteria. This could have been due to lack of selection mechanisms prevailing in the rich environment of the host.",
    "The National Archives Building in downtown Washington holds record collections such as all existing federal census records, ships' passenger lists, military unit records from the American Revolution to the Philippine–American War, records of the Confederate government, the Freedmen's Bureau records, and pension and land records.",
    "Standard 35mm photographic film used for cinema projection has a much higher image resolution than HDTV systems, and is exposed and projected at a rate of 24 frames per second (frame/s). To be shown on standard television, in PAL-system countries, cinema film is scanned at the TV rate of 25 frame/s, causing a speedup of 4.1 percent, which is generally considered acceptable. In NTSC-system countries, the TV scan rate of 30 frame/s would cause a perceptible speedup if the same were attempted, and the necessary correction is performed by a technique called 3:2 Pulldown: Over each successive pair of film frames, one is held for three video fields (1/20 of a second) and the next is held for two video fields (1/30 of a second), giving a total time for the two frames of 1/12 of a second and thus achieving the correct average film frame rate.",
    "Maria Deraismes was initiated into Freemasonry in 1882, then resigned to allow her lodge to rejoin their Grand Lodge. Having failed to achieve acceptance from any masonic governing body, she and Georges Martin started a mixed masonic lodge that actually worked masonic ritual. Annie Besant spread the phenomenon to the English speaking world. Disagreements over ritual led to the formation of exclusively female bodies of Freemasons in England, which spread to other countries. Meanwhile, the French had re-invented Adoption as an all-female lodge in 1901, only to cast it aside again in 1935. The lodges, however, continued to meet, which gave rise, in 1959, to a body of women practising continental Freemasonry.",
    "Excavation of the foundations began in November 1906, with an average of 275 workers during the day shift and 100 workers during the night shift. The excavation was required to be completed in 120 days. To remove the spoils from the foundation, three temporary wooden platforms were constructed to street level. Hoisting engines were installed to place the beams for the foundation, while the piers were sunk into the ground under their own weight. Because of the lack of space in the area, the contractors' offices were housed beneath the temporary platforms. During the process of excavation, the Gilsey Building's foundations were underpinned or shored up, because that building had relatively shallow foundations descending only 18 feet (5.5 m) below Broadway.",
    "Dopamine consumed in food cannot act on the brain, because it cannot cross the blood–brain barrier. However, there are also a variety of plants that contain L-DOPA, the metabolic precursor of dopamine. The highest concentrations are found in the leaves and bean pods of plants of the genus Mucuna, especially in Mucuna pruriens (velvet beans), which have been used as a source for L-DOPA as a drug. Another plant containing substantial amounts of L-DOPA is Vicia faba, the plant that produces fava beans (also known as 'broad beans'). The level of L-DOPA in the beans, however, is much lower than in the pod shells and other parts of the plant. The seeds of Cassia and Bauhinia trees also contain substantial amounts of L-DOPA.",
]

questions = [
    "What was the predominant theory of reality that Whitehead opposed?",
    "Why do the gills on the Psilocybe pelliculosa mushroom darken as they mature?",
    "user: How do I provision my AT&T SIM card?",
    "Why did chief astronaut Deke Slayton choose Charles Bassett for Gemini 9, according to Slayton's autobiography?",
    "What is the main alteration in an endosymbiont when it adapts to a host?",
    "What's the earliest war The National Archives Building has military unit records for",
    "To be shown on SDTV in PAL-system countries, at what rate is cinema film scanned?",
    "What year was the all-female masonic lodge cast aside?",
    "Why did the Gilsey Building have underpinned and shored up foundations?",
    "Why can dopamine consumed in food not act on the brain?",
]

In [ ]:
examples = pd.DataFrame(
    {
        "questions": questions,
        "context": contexts,
    }
)
examples.head()

In [ ]:
examples["response_a"] = examples.apply(
    lambda x: self_model_generate(
        x["questions"], x["context"], max_tokens=2500, temperature=0.5
    ),
    axis=1,
)
examples.head()

In [ ]:
examples["response_b"] = examples.apply(
    lambda x: maas_generate(
        x["questions"], x["context"], max_tokens=2500, temperature=0.5
    ),
    axis=1,
)
examples.head()

#### Upload your dataset to Cloud Storage

Finally, we upload our evaluation dataset to Cloud Storage to be used as input for AutoSxS.

In [ ]:
examples.to_json(f"{BUCKET_URI}/evaluation_dataset.json", orient="records", lines=True)

### Create and run AutoSxS job

In order to run AutoSxS, we need to define a `autosxs_pipeline` job with the following parameters.

More details of the AutoSxS pipeline configuration can be found [here](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/preview/model_evaluation.html#preview.model_evaluation.autosxs_pipeline).

First, compile the AutoSxS pipeline locally.

In [ ]:
template_uri = "pipeline.yaml"
compiler.Compiler().compile(
    pipeline_func=model_evaluation.autosxs_pipeline,
    package_path=template_uri,
)

The following code starts a Vertex Pipeline job, viewable from the Vertex UI. This pipeline job will take ~15 mins. This pipeline is made for batch prediction at a much larger scale than this example, so the time won't scale up linearly if there were thousands of Q&A pairs.

In [ ]:
display_name = f"autosxs-qa-{generate_uuid()}"
context_column = "context"
question_column = "questions"
response_column_a = "response_a"
response_column_b = "response_b"

parameters = {
    "evaluation_dataset": BUCKET_URI + "/evaluation_dataset.json",
    "id_columns": [question_column],
    "autorater_prompt_parameters": {
        "inference_context": {"column": context_column},
        "inference_instruction": {"column": question_column},
    },
    "task": "question_answering",
    "response_column_a": response_column_a,
    "response_column_b": response_column_b,
}

job = aiplatform.PipelineJob(
    job_id=display_name,
    display_name=display_name,
    pipeline_root=BUCKET_URI + "/pipeline",
    template_path=template_uri,
    parameter_values=parameters,
    enable_caching=False,
    project=PROJECT_ID,
    location=LOCATION,
)
job.run()

### Get the judgments and AutoSxS metrics
Next, you can review judgments from the completed AutoSxS job.

In [ ]:
for details in job.task_details:
    if details.task_name == "online-evaluation-pairwise":
        break

judgments_uri = details.outputs["judgments"].artifacts[0].uri
judgments_df = pd.read_json(judgments_uri, lines=True)
judgments_df.head()

You can also review AutoSxS metrics computed from the judgments.

You can find more details of AutoSxS metrics [here](https://cloud.google.com/vertex-ai/generative-ai/docs/models/side-by-side-eval#aggregate-metrics).

In [ ]:
for details in job.task_details:
    if details.task_name == "model-evaluation-text-generation-pairwise":
        break
pd.DataFrame([details.outputs["autosxs_metrics"].artifacts[0].metadata])

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

Set `delete_bucket` to **True** to delete the Cloud Storage bucket.

In [ ]:
delete_pipeline_job = False  # @param {type:"boolean"}
delete_bucket = False  # @param {type:"boolean"}

if delete_pipeline_job:
    job.delete()

if delete_bucket:
    ! gsutil rm -r gs://{BUCKET_NAME}